# Sleep Scoring Model Single File
Training a model on manual sleep scoring for a single file.

In [3]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)


# Load data

In [4]:
input_file = "C:\\dev\\play\\brainwave-data\\2024-09-01-21-11-15\\raw.fif"
input_file_without_ext = os.path.splitext(input_file)[0]

In [5]:
import run_feature_pipeline
yasa_df = run_feature_pipeline.cached_pipeline(log, input_file)
yasa_df

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,Fpz-M1_eeg_std,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,YASAProbablyAwake,DefinitelyAwake,ProbablyAwake,ProbablySleep
0,W,0.998098,0,2024-09-01 21:11:15.609066010+01:00,Fpz-M1,W,0.998098,0,1.924170e-09,0.674103,...,0.000061,0.408875,0.114821,0.143303,-0.031615,0.281790,False,False,False,False
1,W,0.998786,1,2024-09-01 21:11:45.609066010+01:00,Fpz-M1,W,0.998786,0,1.538265e-09,0.742847,...,0.000072,0.457701,0.133877,0.091520,-0.061840,0.096633,True,False,True,False
2,W,0.998694,2,2024-09-01 21:12:15.609066010+01:00,Fpz-M1,W,0.998694,0,3.222337e-09,0.804278,...,0.000268,0.498781,0.366230,0.098994,-0.076986,0.052729,True,False,True,False
3,W,0.997929,3,2024-09-01 21:12:45.609066010+01:00,Fpz-M1,W,0.997929,0,1.632114e-09,0.837591,...,0.000069,0.495865,0.310115,0.090746,-0.092663,0.016031,True,False,True,False
4,W,0.999127,4,2024-09-01 21:13:15.609066010+01:00,Fpz-M1,W,0.999127,0,1.673459e-09,0.880257,...,0.000053,0.505479,0.303755,0.068711,-0.108333,-0.117325,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,W,0.974482,1089,2024-09-02 06:15:45.609066010+01:00,Fpz-M1,W,0.974482,0,5.987828e-11,0.149338,...,0.000009,0.731613,0.456701,0.136710,0.079478,-0.051760,True,False,True,False
1090,W,0.989291,1090,2024-09-02 06:16:15.609066010+01:00,Fpz-M1,W,0.989291,0,6.303612e-11,0.172236,...,0.000009,0.854302,0.230547,0.291684,0.105994,0.424371,True,False,True,False
1091,W,0.911434,1091,2024-09-02 06:16:45.609066010+01:00,Fpz-M1,W,0.911434,0,1.675323e-10,0.204585,...,0.000454,1.021654,0.320756,0.128080,0.077381,0.643232,True,False,True,False
1092,W,0.988893,1092,2024-09-02 06:17:15.609066010+01:00,Fpz-M1,W,0.988893,0,3.977211e-09,0.252664,...,0.000230,1.151942,0.510719,0.006493,0.021929,0.263471,True,False,True,False


In [6]:
import json
import pandas as pd

json_file_path = input_file.replace('.fif', '.scorings.json')
with open(json_file_path, 'r') as file:
    data = json.load(file)
df = pd.json_normalize(data, 'scorings', errors='ignore')
df['tags'] = df['tags'].apply(lambda x: [tag['tag'] for tag in x] if isinstance(x, list) else [])
df.set_index('epochIndex', inplace=True)
df.rename(columns={'stage': 'ManualStage'}, inplace=True)
df

,scoredAt,ManualStage,tags
epochIndex,,,
0,2024-09-02T17:12:24.695Z,Wake,[Blinks]
1,2024-09-02T17:12:24.791Z,Wake,[Blinks]
2,2024-09-02T17:12:26.099Z,Wake,[Blinks]
3,2024-09-02T17:12:57.334Z,Wake,[Blinks]
4,2024-09-02T17:13:03.132Z,Wake,[Blinks]
...,...,...,...
1089,2024-09-02T18:35:34.392Z,Unsure,[]
1090,2024-09-02T18:35:35.059Z,Unsure,[]
1091,2024-09-02T18:35:40.593Z,Unsure,[MicrowakingComplete]


In [7]:
joined = yasa_df.join(df)
joined

,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,...,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,YASAProbablyAwake,DefinitelyAwake,ProbablyAwake,ProbablySleep,scoredAt,ManualStage,tags
0,W,0.998098,0,2024-09-01 21:11:15.609066010+01:00,Fpz-M1,W,0.998098,0,1.924170e-09,0.674103,...,0.143303,-0.031615,0.281790,False,False,False,False,2024-09-02T17:12:24.695Z,Wake,[Blinks]
1,W,0.998786,1,2024-09-01 21:11:45.609066010+01:00,Fpz-M1,W,0.998786,0,1.538265e-09,0.742847,...,0.091520,-0.061840,0.096633,True,False,True,False,2024-09-02T17:12:24.791Z,Wake,[Blinks]
2,W,0.998694,2,2024-09-01 21:12:15.609066010+01:00,Fpz-M1,W,0.998694,0,3.222337e-09,0.804278,...,0.098994,-0.076986,0.052729,True,False,True,False,2024-09-02T17:12:26.099Z,Wake,[Blinks]
3,W,0.997929,3,2024-09-01 21:12:45.609066010+01:00,Fpz-M1,W,0.997929,0,1.632114e-09,0.837591,...,0.090746,-0.092663,0.016031,True,False,True,False,2024-09-02T17:12:57.334Z,Wake,[Blinks]
4,W,0.999127,4,2024-09-01 21:13:15.609066010+01:00,Fpz-M1,W,0.999127,0,1.673459e-09,0.880257,...,0.068711,-0.108333,-0.117325,True,False,True,False,2024-09-02T17:13:03.132Z,Wake,[Blinks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,W,0.974482,1089,2024-09-02 06:15:45.609066010+01:00,Fpz-M1,W,0.974482,0,5.987828e-11,0.149338,...,0.136710,0.079478,-0.051760,True,False,True,False,2024-09-02T18:35:34.392Z,Unsure,[]
1090,W,0.989291,1090,2024-09-02 06:16:15.609066010+01:00,Fpz-M1,W,0.989291,0,6.303612e-11,0.172236,...,0.291684,0.105994,0.424371,True,False,True,False,2024-09-02T18:35:35.059Z,Unsure,[]
1091,W,0.911434,1091,2024-09-02 06:16:45.609066010+01:00,Fpz-M1,W,0.911434,0,1.675323e-10,0.204585,...,0.128080,0.077381,0.643232,True,False,True,False,2024-09-02T18:35:40.593Z,Unsure,[MicrowakingComplete]
1092,W,0.988893,1092,2024-09-02 06:17:15.609066010+01:00,Fpz-M1,W,0.988893,0,3.977211e-09,0.252664,...,0.006493,0.021929,0.263471,True,False,True,False,2024-09-02T18:35:42.863Z,Wake,[]


In [8]:
joined['tags'].value_counts()

tags
[]                                                 872
[Possible non-wake disturbance]                    104
[Microwaking]                                       29
[MicrowakingComplete]                               24
[MicrowakingEnd]                                    16
[Weak deep]                                         14
[Blinks]                                            12
[Microwaking, MicrowakingEnd]                        7
[Strong deep]                                        5
[MicrowakingEnd, Possible non-wake disturbance]      3
[Unusual]                                            1
[Possible non-wake disturbance, Weak deep]           1
[Microwaking, Possible non-wake disturbance]         1
[Uniform]                                            1
Name: count, dtype: int64

# Preparing data for model

In [9]:
joined['ManualStage'].value_counts()

ManualStage
Non-Deep          658
Deep              165
Wake              157
Ambiguous Deep     56
Unsure             54
Name: count, dtype: int64

In [10]:
import manual_sleep_scoring_1
prepared_df, X, y = manual_sleep_scoring_1.pipeline(joined)

display(prepared_df)
display(X)
display(y)

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,0.408875,0.114821,0.143303,-0.031615,0.281790,0.0,0.0,0.0,0.0,1.0
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,0.457701,0.133877,0.091520,-0.061840,0.096633,0.0,0.0,0.0,0.0,1.0
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,0.498781,0.366230,0.098994,-0.076986,0.052729,0.0,0.0,0.0,0.0,1.0
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,0.495865,0.310115,0.090746,-0.092663,0.016031,0.0,0.0,0.0,0.0,1.0
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,0.505479,0.303755,0.068711,-0.108333,-0.117325,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,5.987828e-11,0.149338,0.181180,0.243420,0.891423,0.906780,1.780555,0.717088,1.047208,0.053974,...,0.731613,0.456701,0.136710,0.079478,-0.051760,0.0,0.0,0.0,1.0,0.0
1090,6.303612e-11,0.172236,0.127809,0.361016,0.961282,1.792029,1.237698,0.730596,1.284927,0.054653,...,0.854302,0.230547,0.291684,0.105994,0.424371,0.0,0.0,0.0,1.0,0.0
1091,1.675323e-10,0.204585,-0.026512,0.154846,0.921318,2.164861,1.208976,0.721860,1.167191,1.215822,...,1.021654,0.320756,0.128080,0.077381,0.643232,0.0,0.0,0.0,1.0,0.0
1092,3.977211e-09,0.252664,0.273996,0.003787,0.833743,1.267842,0.583230,0.686293,0.803938,0.879373,...,1.151942,0.510719,0.006493,0.021929,0.263471,0.0,0.0,0.0,0.0,1.0


,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_sigma_p2min_norm,Fpz-M1_eeg_skew,Fpz-M1_eeg_skew_c7min_norm,Fpz-M1_eeg_skew_p2min_norm,Fpz-M1_eeg_std,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,-0.286546,-0.041498,-0.292168,-0.168319,0.000061,0.408875,0.114821,0.143303,-0.031615,0.281790
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,-0.314349,-1.054456,-0.295245,-0.300345,0.000072,0.457701,0.133877,0.091520,-0.061840,0.096633
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,-0.298760,0.206473,-0.253102,-0.234790,0.000268,0.498781,0.366230,0.098994,-0.076986,0.052729
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,-0.271386,-0.612750,-0.229889,-0.255400,0.000069,0.495865,0.310115,0.090746,-0.092663,0.016031
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,-0.278903,-0.476831,-0.239482,-0.283770,0.000053,0.505479,0.303755,0.068711,-0.108333,-0.117325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,5.987828e-11,0.149338,0.181180,0.243420,0.891423,0.906780,1.780555,0.717088,1.047208,0.053974,...,0.166258,-0.141322,-0.159096,-0.223251,0.000009,0.731613,0.456701,0.136710,0.079478,-0.051760
1090,6.303612e-11,0.172236,0.127809,0.361016,0.961282,1.792029,1.237698,0.730596,1.284927,0.054653,...,0.497499,-0.034740,-0.153467,-0.196748,0.000009,0.854302,0.230547,0.291684,0.105994,0.424371
1091,1.675323e-10,0.204585,-0.026512,0.154846,0.921318,2.164861,1.208976,0.721860,1.167191,1.215822,...,0.715273,1.139728,-0.145289,-0.121289,0.000454,1.021654,0.320756,0.128080,0.077381,0.643232
1092,3.977211e-09,0.252664,0.273996,0.003787,0.833743,1.267842,0.583230,0.686293,0.803938,0.879373,...,0.406942,-1.350490,-0.146132,-0.182710,0.000230,1.151942,0.510719,0.006493,0.021929,0.263471


,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
1089,0.0,0.0,0.0,1.0,0.0
1090,0.0,0.0,0.0,1.0,0.0
1091,0.0,0.0,0.0,1.0,0.0
1092,0.0,0.0,0.0,0.0,1.0


In [16]:
y.value_counts()

ManualStage_Ambiguous Deep  ManualStage_Deep  ManualStage_Non-Deep  ManualStage_Unsure  ManualStage_Wake
0.0                         0.0               1.0                   0.0                 0.0                 658
                            1.0               0.0                   0.0                 0.0                 165
                            0.0               0.0                   0.0                 1.0                 157
1.0                         0.0               0.0                   0.0                 0.0                  56
0.0                         0.0               0.0                   1.0                 0.0                  54
Name: count, dtype: int64

In [11]:
full_index_range = pd.RangeIndex(start=prepared_df.index.min(), stop=prepared_df.index.max() + 1)
unscored_indexes = full_index_range.difference(prepared_df.index)
unscored_indexes

Index([368, 726, 914, 917], dtype='int64')

In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train

,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake
573,0.0,0.0,1.0,0.0,0.0
810,0.0,0.0,1.0,0.0,0.0
835,0.0,0.0,1.0,0.0,0.0
918,0.0,0.0,0.0,0.0,1.0
331,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
330,0.0,0.0,1.0,0.0,0.0
467,0.0,1.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,1.0
1048,0.0,0.0,1.0,0.0,0.0


# Training multiple LGBM models

In [ ]:
# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define parameters
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': target_cols.__len__(),
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
bst = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, test_data])


In [ ]:
train_data_list = [lgb.Dataset(X_train, label=y_train[col]) for col in target_cols]
test_data_list = [lgb.Dataset(X_test, label=y_test[col], reference=train_data) for col, train_data in zip(target_cols, train_data_list)]

# Define parameters
params = {
    'objective': 'binary',  # Use binary classification for each label
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model for each target column
models = [lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, test_data]) for train_data, test_data in zip(train_data_list, test_data_list)]

# Predict probabilities for each target column
y_pred_prob_list = [model.predict(X_test) for model in models]

# Combine predictions into a DataFrame
y_pred_prob_df = pd.DataFrame({col: y_pred_prob for col, y_pred_prob in zip(target_cols, y_pred_prob_list)})

y_pred_prob_df

In [ ]:
# Preserve the original index
original_index = X_test.index

# Predict probabilities for each target column
y_pred_prob_list = [model.predict(X_test) for model in models]

# Combine predictions into a DataFrame
y_pred_prob_df = pd.DataFrame({col: y_pred_prob for col, y_pred_prob in zip(target_cols, y_pred_prob_list)}, index=original_index)

# Convert probabilities to binary predictions
y_pred_list = [(y_pred_prob > 0.5).astype(int) for y_pred_prob in y_pred_prob_list]

# Combine predictions and true labels into a DataFrame
results_df = pd.DataFrame(index=original_index)

for col, y_pred, y_true in zip(target_cols, y_pred_list, y_test.values.T):
    results_df[f'{col}_pred'] = y_pred
    results_df[f'{col}_true'] = y_true

results_df

    # Evaluating the model

In [ ]:
rejoined = joined.join(results_df)
# cols_to_drop = [col for col in rejoined.columns if '_true' in col]
# rejoined.drop(columns=cols_to_drop, inplace=True)
rejoined

In [ ]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions
y_pred_list = [(y_pred_prob > 0.5).astype(int) for y_pred_prob in y_pred_prob_list]

# Calculate accuracy for each target column
accuracy_scores = {col: accuracy_score(y_test[col], y_pred) for col, y_pred in zip(target_cols, y_pred_list)}

accuracy_scores

In [ ]:
import pandas as pd

# Convert probabilities to binary predictions
y_pred_list = [(y_pred_prob > 0.5).astype(int) for y_pred_prob in y_pred_prob_list]

# Combine predictions and true labels into a DataFrame
results_df = pd.DataFrame()

for col, y_pred, y_true in zip(target_cols, y_pred_list, y_test.values.T):
    results_df[f'{col}_pred'] = y_pred
    results_df[f'{col}_true'] = y_true

results_df

In [ ]:
results_df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.clf()

# Extract true and predicted labels
stages = ['Ambiguous Deep', 'Deep', 'Non-Deep', 'Unsure', 'Wake']
true_labels = [f'ManualStage_{stage}_true' for stage in stages]
pred_labels = [f'ManualStage_{stage}_pred' for stage in stages]

# Plot confusion matrix for each stage
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.flatten()

for i, stage in enumerate(stages):
    y_true = results_df[true_labels[i]]
    y_pred = results_df[pred_labels[i]]

    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', ax=axes[i], cmap='Blues')
    axes[i].set_title(f'Confusion Matrix for {stage}')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')

# Remove the last empty subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

# Saving the model

In [ ]:
import joblib

# Define the file path where the models will be saved
models_file_path = input_file_without_ext + ".manual_sleep_scoring_model.pkl"

# Save the list of trained models
joblib.dump(models, models_file_path)